
Copyright (C) 2024 - 2026 ANSYS, Inc. and/or its affiliates.
SPDX-License-Identifier: Apache-2.0


Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Import CAD and create ISAR 2D setup

This example demonstrates how to use the ``ToolkitBackend`` class.
It initiates AEDT through PyAEDT, opens an SBR+ design, creates the setup, and analyzes it.

## Perform required imports

In [1]:
from pathlib import Path
import shutil
import sys
import tempfile
import time

In [2]:
from ansys.aedt.toolkits.radar_explorer.backend.api import ToolkitBackend
from ansys.aedt.toolkits.radar_explorer.rcs_visualization import MonostaticRCSData
from ansys.aedt.toolkits.radar_explorer.rcs_visualization import MonostaticRCSPlotter

## Set AEDT version

In [3]:
aedt_version = "2025.1"

## Set non-graphical mode

In [4]:
non_graphical = True

## Set number of cores

In [5]:
cores = 4

## Create temporary directory

In [6]:
temp_dir = tempfile.TemporaryDirectory(suffix="_ansys")

## Get example project

In [7]:
car_original = r"example_models\geometries\car_stl.stl"
car = Path(temp_dir.name) / "car.stl"
shutil.copy(car_original, car)

WindowsPath('C:/Users/ansys/AppData/Local/Temp/tmprwmf4sw__ansys/car.stl')

## Initialize toolkit

In [8]:
toolkit_api = ToolkitBackend()

## Get toolkit properties

In [9]:
properties_from_backend = toolkit_api.get_properties()

## Set properties

Set non-graphical mode.

In [10]:
set_properties = {"non_graphical": non_graphical, "aedt_version": aedt_version}
flag_set_properties, msg_set_properties = toolkit_api.set_properties(set_properties)

INFO - Updating internal properties.


## Initialize AEDT

Launch a new AEDT session in a thread.

In [11]:
thread_msg = toolkit_api.launch_thread(toolkit_api.launch_aedt)

PyAEDT INFO: Python version 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.24.1.


PyAEDT INFO: Initializing new Desktop session.


## Wait for the toolkit thread to be idle

Wait for the toolkit thread to be idle and ready to accept a new task.

In [12]:
idle = toolkit_api.wait_to_be_idle()
if not idle:
    print("AEDT not initialized.")
    sys.exit()

Exception in thread Toolkit_Thread:
Traceback (most recent call last):
  File "C:\actions-runner\_work\_tool\Python\3.12.10\x64\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\actions-runner\_work\_tool\Python\3.12.10\x64\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\actions-runner\_work\ansys-aedt-toolkits-radar-explorer\ansys-aedt-toolkits-radar-explorer\.venv\Lib\site-packages\ansys\aedt\toolkits\common\backend\thread_manager.py", line 55, in process_exe
    process(*args)
  File "C:\actions-runner\_work\ansys-aedt-toolkits-radar-explorer\ansys-aedt-toolkits-radar-explorer\.venv\Lib\site-packages\ansys\aedt\toolkits\common\backend\api.py", line 429, in launch_aedt
    self.desktop = ansys.aedt.core.Desktop(**desktop_args)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: __init__() should return None, not 'bool'


ERROR - Toolkit has crashed and is not functional.


## Connect design

Connect an existing design or create a new design.

In [13]:
toolkit_api.connect_design("HFSS")

ERROR - Process ID is not defined.


False

## Import CAD

In [14]:
toolkit_api.properties.cad.input_file = [car]
toolkit_api.properties.cad.material = ["pec"]
toolkit_api.properties.cad.position = [[0.0, 0.0, 0.0]]

## Set calculation type

In [15]:
new_properties = {"calculation_type": "2D ISAR"}

In [16]:
flag3, msg3 = toolkit_api.set_properties(new_properties)

INFO - Updating internal properties.


In [17]:
new_properties = {"ray_density": 0.1, "ffl": True, "num_cores": cores}
flag4, msg4 = toolkit_api.set_properties(new_properties)

INFO - Updating internal properties.


## Set ISAR 2D properties

In [18]:
new_properties = {
    "range_max_az": 14.3,
    "range_res_az": 0.15,
    "range_max": 15.0,
    "range_res": 0.15,
    "sim_freq_lower": 9.5e9,
    "sim_freq_upper": 10.5e9,
}
flag5, msg5 = toolkit_api.set_properties(new_properties)

INFO - Updating internal properties.


In [19]:
properties1 = toolkit_api.get_properties()

C:\actions-runner\_work\ansys-aedt-toolkits-radar-explorer\ansys-aedt-toolkits-radar-explorer\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='input_file', input_value=WindowsPath('C:/Users/ans...rwmf4sw__ansys/car.stl'), input_type=WindowsPath])
  return self.__pydantic_serializer__.to_python(


In [20]:
toolkit_api.update_isar_2d_properties(range_is_system=False, azimuth_is_system=False)

Check how ``aspect_ang_phi`` and ``num_phi`` has changed.

In [21]:
properties2 = toolkit_api.get_properties()

## Connect design and load project information

In [22]:
toolkit_api.launch_aedt()

PyAEDT INFO: Python version 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.24.1.


PyAEDT INFO: Initializing new Desktop session.


TypeError: __init__() should return None, not 'bool'

## Insert CAD

In [23]:
toolkit_api.insert_cad_sbr()

ERROR - Process ID is not defined.


ERROR - Toolkit cannot connect to AEDT.


INFO - AEDT is released.


False

## Assign excitation

In [24]:
v_plane_wave = toolkit_api.add_plane_wave(name="IncWaveVpol", polarization="Vertical")

ERROR - Process ID is not defined.


ERROR - Toolkit can not connect to AEDT.


INFO - AEDT is released.


## Create setup

In [25]:
setup_name = toolkit_api.add_setup()

ERROR - Process ID is not defined.


ERROR - Toolkit can not connect to AEDT.


INFO - AEDT is released.


## Get toolkit properties

In [26]:
properties = toolkit_api.get_properties()

## Analyze

In [27]:
toolkit_api.analyze()
toolkit_api.save_project()

ERROR - Process ID is not defined.


ERROR - Toolkit can not connect to AEDT.


INFO - AEDT is released.


ERROR - Process ID is not defined.


ERROR - Project is not saved.


False

## Get RCS data

In [28]:
rcs_metadata_vv = toolkit_api.export_rcs(v_plane_wave, "ComplexMonostaticRCSTheta", encode=False)

ERROR - Process ID is not defined.


ERROR - Toolkit cannot connect to AEDT.


INFO - AEDT is released.


## Save and release AEDT

In [29]:
toolkit_api.release_aedt(True, True)

ERROR - Process ID is not defined.


INFO - AEDT is released.


True

## Load RCS data

In [30]:
rcs_data_vv = MonostaticRCSData(rcs_metadata_vv)

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'bool'

## Load RCS plotter

In [31]:
rcs_data_vv_plotter = MonostaticRCSPlotter(rcs_data_vv)

NameError: name 'rcs_data_vv' is not defined

## Set ISAR 2D

In [32]:
rcs_data_vv_plotter.plot_isar_2d()

NameError: name 'rcs_data_vv_plotter' is not defined

## Plot ISAR

In [33]:
rcs_data_vv_plotter.clear_scene()
rcs_data_vv_plotter.add_isar_2d()
rcs_data_vv_plotter.plot_scene()

NameError: name 'rcs_data_vv_plotter' is not defined

In [34]:
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

## Clean temporary directory

In [35]:
temp_dir.cleanup()